### Create S3 Bucket

In [24]:
import logging
import boto3
from botocore.exceptions import ClientError
import json

ACCESS_KEY = '********************'
SECRET_KEY = '****************************************'

def create_bucket(bucket_name, location, key, value):
    """
    https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.create_bucket
    """
    s3 = boto3.client('s3',
                      aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY,
#                       region_name=location
                     
                     )
    
    try:
        response1 = s3.create_bucket(ACL='private',
                                    Bucket=bucket_name,
                                    CreateBucketConfiguration={'LocationConstraint': location}
                                   )
        tag={'TagSet':[{'Key': key, 'Value': value}]}
        response2 = s3.put_bucket_tagging(Bucket=bucket_name, Tagging=tag)
        
        logging.info(json.dumps(response1, indent=4))
        logging.info(json.dumps(response2, indent=4))
    except ClientError as e:
        logging.error(e)
        print('error')
        
    return response1

bucket_name = 'nithin-test123'
location = 'ap-south-1'
key = 'flag'
value = 'bucketflag'

# create_bucket(bucket_name, location, key, value)

### Create Lambda function

In [44]:
import logging
import boto3
from botocore.exceptions import ClientError
import json

ACCESS_KEY = '********************'
SECRET_KEY = '****************************************'

def create_lambda(lambda_name,location, S3_bucket, S3_key, key, value):
    """
    https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lambda.html
    """
    
    lambda_ = boto3.client('lambda', region_name=location,
                           aws_access_key_id=ACCESS_KEY,
                           aws_secret_access_key=SECRET_KEY)
    try:

        response1 =  lambda_.create_function(FunctionName=lambda_name,
                                Runtime='python3.6',
                                Role='arn:aws:iam::453890640763:role/lambda-role',
                                Handler='function.lambda_handler',
                                Code={
                                    'S3Bucket':S3_bucket ,
                                    'S3Key': S3_key,
                                    }
                               )
        response2 = lambda_.tag_resource(
                                Resource=response1['FunctionArn'],
                                Tags={
                                    key : value
                                }
                            )
        
        logging.info(json.dumps(response1, indent=4))
        logging.info(json.dumps(response2, indent=4))
        
    except ClientError as e:
        logging.error(e)
        
    return response1
        
lambda_name = 'delete_this'
location = 'ap-south-1'
key = 'flag'
value = 'lambdaflag'
S3_bucket = 'sannzay2' #bucket name
S3_key = 'first/function.zip'

# create_lambda(lambda_name,location, S3_bucket, S3_key, key, value)

## Create Resource group

In [8]:
import logging
import boto3
from botocore.exceptions import ClientError
import json

ACCESS_KEY = '********************'
SECRET_KEY = '****************************************'


def create_resource_group(rg_name,location,json_):
    
    resource_group = boto3.client('resource-groups', region_name=location, 
                        aws_access_key_id=ACCESS_KEY,
                        aws_secret_access_key=SECRET_KEY)
    try:
        
        response = resource_group.create_group(Name=rg_name,
                                               ResourceQuery={
                                                   'Type': 'TAG_FILTERS_1_0',
                                                   "Query": json.dumps(json_)
                                                   }
                                              )
        logging.info(json.dumps(response, indent=4))
    except ClientError as e:
        logging.error(e)


location = 'ap-south-1'
rg_name = 'abc'
key= 'abcXYZ'
value = 'abcvalue' 
json_ = {}
json_['ResourceTypeFilters'] = ['AWS::S3::Bucket']
json_['TagFilters'] = [{'Key': key, 'Values': [value]}]


# create_resource_group(rg_name,location,json_)

## Attach S3 trigger to Lambda

In [43]:
import boto3

ACCESS_KEY = '********************'
SECRET_KEY = '****************************************'


def create_s3_trigger(bucket_name, lambda_arn, location):
    
    #lambda1 and s3 are the client objects of lambda and s3_bucket respectively
    s3 = boto3.client('s3',
                    aws_access_key_id=ACCESS_KEY,
                    aws_secret_access_key=SECRET_KEY)
    
    lambda_ = boto3.client('lambda', region_name=location,
                       aws_access_key_id=ACCESS_KEY,
                       aws_secret_access_key=SECRET_KEY)
    
    response1 = lambda_.add_permission(FunctionName=lambda_arn,
                                       StatementId=bucket_name+'response1-id-1',
                                       Action='lambda:InvokeFunction',
                                       Principal='s3.amazonaws.com',
                                       SourceArn='arn:aws:s3:::'+bucket_name
                                      )
    response2 = lambda_.get_policy(FunctionName=lambda_arn)
    
    #s3 is s3 bucket boto3 client
    response3 = s3.put_bucket_notification_configuration(
                                    Bucket=bucket_name,
                                    NotificationConfiguration= {'LambdaFunctionConfigurations':[{'LambdaFunctionArn': lambda_arn, 'Events': ['s3:ObjectCreated:*']}]})

location = 'ap-south-1'
bucket_name = 'sannzay3' #bucket name

# lambda_response = create_lambda()
# lambda_arn = lambda_response['FunctionArn']

# create_s3_trigger(bucket_name, lambda_arn, location)

## Attach sqs trigger to Lambda

In [ ]:
import boto3

ACCESS_KEY = '********************'
SECRET_KEY = '****************************************'


def create_sqs_trigger(sqs_arn, lambda_arn, location):
    
    lambda_ = boto3.client('lambda', region_name=location,
                       aws_access_key_id=ACCESS_KEY,
                       aws_secret_access_key=SECRET_KEY)
    
    response1 = lambda_.create_event_source_mapping(
    EventSourceArn=sqs_arn, #this sqs arn can be replaced with kinesis or dynamoDB arn
    FunctionName=lambda_arn,
    Enabled=False
#     BatchSize=123
#     StartingPosition='TRIM_HORIZON'|'LATEST'|'AT_TIMESTAMP',
#     StartingPositionTimestamp=datetime(2015, 1, 1)
    )
    

location = 'ap-south-1'
lambda_response = create_lambda()
sqs_response = create_sqs()
sqs_arn = sqs_response['FunctionArn']
lambda_arn = lambda_response['FunctionArn']

# create_sqs_trigger(sqs_arn, lambda_arn, location)